# Predicting performance direction of Premier League Footballers using 'Fantasy Football' data and Machine learning

Using a LTSM neural network trained with data collected from 3 seasons of Premier League Football games, a predictive model of player performance was made. By 'direction of performance,' it is simply meant whether a player performed better or worse in the next matched player relative to the previous week, i.e. a simple binary choice. Thus, a categorical model is used here.

Below the construction of this model is outlined.

### Importing and normalizing data
The data being used to train the model has been collected from the Official Premier League online game 'Fantasy Premier League." This online game tracks multiple performance attributes for the sport, such as number of shots or number of fouls committed by each player every week, and based on these variables gives each player a score every week. The score that each player gets depends primarily on the number of bonus points ('bps') they recieve, which is determined by set rules- for example 4 bps if they score a goal, 2 bps for playing 90 minutes (the length of a premier league football match) or -2 points if the player receives a red card. These rules are not of direct consequence for the analysis but the full list is found here- https://fantasy.premierleague.com/help/rules.

The number of bps awarded to a given player additionally depends on how well that player performs relative to other players in the same match for that week. Thus, bps, and total points awarded per week, are sufficiently random variables that prediction of them is a problem well suited for machine learning, given the large number of variables which may impact these dependent variables.

The first step for the analysis is to import the data, which is seperated into data for each season, and within each season separated into individual .csv files for each player. Each file then has various attribute values recorded for each week (of which there are 38 weeks). We begin by importing the required modules and defining the classification function, which will be used for defining the performance direction:

In [1]:
import pandas as pd
import numpy as np
import os
import datetime

def classify(current, future):
    if float(future) > float(current):
        return 1
    else:
        return 0

Next, the csv files need to be processed. Using the file 'raw_data/2016-17/players/Sergio_Agüero/gw.csv' as an example to show the format of these files, we have:

In [2]:
dat = pd.read_csv("raw_data/2016-17/players/Sergio_Agüero/gw.csv")
dat.columns

Index(['assists', 'attempted_passes', 'big_chances_created',
       'big_chances_missed', 'bonus', 'bps', 'clean_sheets',
       'clearances_blocks_interceptions', 'completed_passes', 'creativity',
       'dribbles', 'ea_index', 'element', 'errors_leading_to_goal',
       'errors_leading_to_goal_attempt', 'fixture', 'fouls', 'goals_conceded',
       'goals_scored', 'ict_index', 'id', 'influence', 'key_passes',
       'kickoff_time', 'kickoff_time_formatted', 'loaned_in', 'loaned_out',
       'minutes', 'offside', 'open_play_crosses', 'opponent_team', 'own_goals',
       'penalties_conceded', 'penalties_missed', 'penalties_saved',
       'recoveries', 'red_cards', 'round', 'saves', 'selected', 'tackled',
       'tackles', 'target_missed', 'team_a_score', 'team_h_score', 'threat',
       'total_points', 'transfers_balance', 'transfers_in', 'transfers_out',
       'value', 'was_home', 'winning_goals', 'yellow_cards'],
      dtype='object')

To determine which variables potentially correlate with the target variables 'bps' and 'total_points,' the variable which the model will predict, use the corr() function:

In [3]:
dat["future_points"] = dat['bps'].shift(+1)
dat.corr()[["bps","future_points"]]

,bps,future_points
assists,0.500451,-0.194994
attempted_passes,0.586816,0.170655
big_chances_created,0.333012,-0.004426
big_chances_missed,0.048586,0.197941
bonus,0.909670,-0.015912
bps,1.000000,-0.013018
clean_sheets,0.282780,0.242436
clearances_blocks_interceptions,0.188060,0.089263
completed_passes,0.614451,0.130888
creativity,0.728404,0.156208


Since the data set being used is large, and given that players in different positions will likely have different dependancies on different variables (for example, Sergio_Agüero is a forward so his bps is potentially more dependent on the variable 'creativity' and less dependent on 'clean_sheets' than a defender would be) we will initially use only the most general variables that have the highest correlation with bps:

In [4]:
variables = ["assists", "attempted_passes", "big_chances_missed", "clean_sheets", "completed_passes",
             "creativity", "dribbles", "goals_conceded", "goals_scored", "ict_index", "key_passes",
            "transfers_in", "transfers_out", "value", "bps", "total_points", "future_points", "round", "minutes"]
dat = dat[variables]
dat.head()

,assists,attempted_passes,big_chances_missed,clean_sheets,completed_passes,creativity,dribbles,goals_conceded,goals_scored,ict_index,key_passes,transfers_in,transfers_out,value,bps,total_points,future_points,round,minutes
0,0,33,0,0,26,29.6,1,1,1,14.3,2,0,0,130,33,9,NaN,1,90
1,0,21,0,0,17,13.3,2,1,2,16.7,1,75648,12138,130,57,13,33.0,2,82
2,0,27,0,0,21,17.5,4,1,0,5.2,1,213510,8885,131,6,2,57.0,3,87
3,0,0,0,0,0,0.0,0,0,0,0.0,0,32042,984971,130,0,0,6.0,4,0
4,0,0,0,0,0,0.0,0,0,0,0.0,0,11435,167808,130,0,0,0.0,5,0


A general function is now required to loop through all the files and select the required columns as shown above. Whilst most files are 38 data entries long, there are some anomolies in length which need to be accounted for, which is done below. 

In particular, some of the .csv files many not be of length 38 because the player they correspond too didn't play for a club in the premier league for the whole season. In this case, the rounds are re-labelled, to provide some consistency in how the data is formatted, which is necessary as later the rounds will be used as an index. N.b. the fact that the round indices may not be correctly labelled (as some players may play for the first half of the season and others the second etc, and this detail is lost in the above function's relabelling) is not relevant, as the network will be trained to analyse short-term patterns, and thus the time of season is not a factor.

In [5]:
def Select_Columns():
    new_directory = f"filtered_data_{datetime.date.today()}"
    os.mkdir(f"{new_directory}")
    rounds_array = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
                    21, 22, 23, 24,25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38]
    
    # Fix incorrect indexing in data:
    years = [[16, 17], [17, 18], [18, 19]]
    for year in years:
        player_list = pd.read_csv(f'raw_data/20{year[0]}-{year[1]}/player_idlist.csv')
        for i in range(len(player_list)):
            try:
                player = pd.read_csv(f'raw_data/20{year[0]}-{year[1]}/players/{player_list.iloc[i][0]}_{player_list.iloc[i][1]}/gw.csv')
                if len(player["round"]) == 38:
                    if player["round"].iloc[-1] != 38:
                        player["round"].iloc[-1] = 38
                difference_length = 38 - len(player["round"])
                if len(player["round"]) == 38:
                    player["round"] = rounds_array
                else:
                    player["round"] = rounds_array[difference_length:]
                player["future_points"] = player['bps'].shift(+1)
                player = player[variables]
                with open(f"{new_directory}/{player_list.iloc[i][0]}_{player_list.iloc[i][1]}_{year[0]}.csv", 'w') as file:
                    player.to_csv(file, header=True, mode='w',index=False)
                        
            except:
                player = pd.read_csv(f'raw_data/20{year[0]}-{year[1]}/players/{player_list.iloc[i][0]}_{player_list.iloc[i][1]}_{player_list.iloc[i][2]}/gw.csv')
                if len(player["round"]) == 38:
                    if player["round"].iloc[-1] != 38:
                        player["round"].iloc[-1] = 38
                difference_length = 38 - len(player["round"])
                if len(player["round"]) == 38:
                    player["round"] = rounds_array
                else:
                    player["round"] = rounds_array[difference_length:]
                player["future_points"] = player['bps'].shift(+1)
                player = player[variables]
                with open(f"{new_directory}/{player_list.iloc[i][0]}_{player_list.iloc[i][1]}_{year[0]}.csv", 'w') as file:
                    player.to_csv(file, header=True, mode='w',index=False)
                        
    return new_directory

working_dir = Select_Columns()

Next, normalize and then structure the data in preparation for passing onto the LTSM:

In [6]:
def Normalise(passed_directory):
    """ Add new column, 'Performance_direction,' to label whether a player performed better
    or worse next week relative to the current week:"""
    
    for filename in os.listdir(passed_directory):
        if filename.endswith(".csv"):
            df = pd.read_csv(f"{passed_directory}/{filename}")
            df["performance_direction"] = list(map(classify, df["future_points"], df["total_points"]))
            with open(f"{passed_directory}/{filename}", 'w') as file:
                df.to_csv(file, header=True, mode='w',index=False)
    
    """ Create a new dict to hold the maxiumum values for each variable, which can then
    be used to normalise the data. The column 'round' is dropped as it is not a variable,
    and 'performance_direction' is added."""
    
    max_values= np.zeros(19)
    variables.remove("round")
    variables.append("performance_direction")
    variables_max_values = dict(zip(variables, max_values))
    """ loop through all files to determine max values"""
    for filename in os.listdir(passed_directory):
        if filename.endswith(".csv"):
            df = pd.read_csv(f"{passed_directory}/{filename}")
            if len(df.index) > 1:
                for var, value in variables_max_values.items():
                    if df.at[df[var].idxmax(axis=1), var] > value:
                        variables_max_values[var] = df.at[df[var].idxmax(axis=1), var]
            else:
                pass
            
    """ loop through a second time to normalise"""
    os.mkdir(f"Normalised_{passed_directory}")
    for filename in os.listdir(passed_directory):
        if filename.endswith(".csv"):
            df = pd.read_csv(f"{passed_directory}/{filename}")
            for var, value in variables_max_values.items():
                df[var]= df[var].div(value)
    
        """ save file for later if there are more than 30 non-zero entries"""
        if df["minutes"][df["minutes"] > 0].count() > 30:
            with open(f"Normalised_{passed_directory}/{filename}", 'w') as file:
                df.to_csv(file, header=True, mode='w', index=False)
    new_directory = f"Normalised_{passed_directory}"
    return variables_max_values, new_directory
    
normlisation_factors, working_dir = Normalise(working_dir)

Now that the data is normalised it must be re-grouped in preparation for the LTSM. The length in the function below refers to the number of periods of data that will be feed into the model at once, i.e. Length = 3 corresponds to 3 weeks of match data. In order for temporal patterns in the data to be abstracted, it is necessary for the order to be retained in each data set, which is what the function deque allows for below.

In [7]:
from collections import deque
import random

In [7]:
from collections import deque
import random

def Sequence_data(passed_directory, length):
    j=0
    SEQ_LEN = length
    sequential_data = []  # this is a list that will CONTAIN the sequences
    for filename in os.listdir(f"{passed_directory}"):
        if filename.endswith(".csv"):
            df = pd.read_csv(f"{passed_directory}/{filename}", index_col="round")
            input_df = df[["transfers_in", "transfers_out",'bps', 'ict_index', 'minutes', 'performance_direction']]  # performance predictior network from 'bps', 'ict_index', 'minutes'
            prev_days = deque(maxlen=SEQ_LEN)  
            for i in input_df.values: 
                prev_days.append([n for n in i[:-1]])  
                if len(prev_days) == SEQ_LEN:  
                    sequential_data.append([np.array(prev_days), i[-1]])  
    random.shuffle(sequential_data)
    #print(np.shape(sequential_data))
    #normal_data = min_max_scaler.fit_transform(np.array(sequential_data))
    return sequential_data


Length = 3
one= Sequence_data(working_dir, Length)


In [8]:
x = []
Y = []
for features, result in one:
    x.append(features)
    Y.append(result)
x = np.array(x, dtype="float32")

In [9]:
import tensorflow as tf
import sklearn
from sklearn import preprocessing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization, SimpleRNN
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ModelCheckpoint
import time

In [20]:
def Network(IN, OUT, TIME_PERIOD, EPOCHS, BATCH_SIZE, LTSM_SHAPE):
 
    length = len(OUT)
    train_x = IN[:int(0.9 * length)]
    validation_x = IN[int(0.9 * length):]
    train_y = OUT[:int(0.9 * length)]
    validation_y = OUT[int(0.9 * length):]

    # Define Network & callback:
    NAME = f"pb_{TIME_PERIOD}_{EPOCHS}_{BATCH_SIZE}_{LTSM_SHAPE}_{time.time()}"
    ternsorboard = TensorBoard(log_dir=f"logs/{NAME}")

    model = Sequential()
    model.add(LSTM(LTSM_SHAPE, input_shape=(train_x.shape[1:]), return_sequences=True))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())  # normalizes activation outputs, same reason you want to normalize your input data.

    model.add(LSTM(LTSM_SHAPE, return_sequences=True))
    model.add(Dropout(0.1))
    model.add(BatchNormalization())

    model.add(LSTM(LTSM_SHAPE))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())

    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))

    model.add(Dense(2, activation='softmax'))

    opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    train_y = np.asarray(train_y)
    validation_y = np.asarray(validation_y)
    history = model.fit(train_x, train_y, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(validation_x, validation_y), callbacks=[ternsorboard])
    print('\nhistory dict:', history.history)

    # Score model
    score = model.evaluate(validation_x, validation_y, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
    # Save model
    model.save(f"models/{NAME}")

EPOCHS, BATCH_SIZE, LTSM_SHAPE = 15, 32, 128

Network(x, Y, Length, EPOCHS, BATCH_SIZE, LTSM_SHAPE)

Train on 57611 samples, validate on 6402 samples
Epoch 1/15
57611/57611 [==============================] - 20s 346us/sample - loss: 0.3251 - accuracy: 0.8886 - val_loss: 0.2733 - val_accuracy: 0.9066
Epoch 2/15
57611/57611 [==============================] - 17s 297us/sample - loss: 0.2686 - accuracy: 0.9060 - val_loss: 0.2786 - val_accuracy: 0.9133
Epoch 3/15
57611/57611 [==============================] - 16s 275us/sample - loss: 0.2493 - accuracy: 0.9128 - val_loss: 0.2404 - val_accuracy: 0.9169
Epoch 4/15
57611/57611 [==============================] - 16s 272us/sample - loss: 0.2515 - accuracy: 0.9115 - val_loss: 0.2394 - val_accuracy: 0.9149
Epoch 5/15
57611/57611 [==============================] - 16s 275us/sample - loss: 0.2408 - accuracy: 0.9138 - val_loss: 0.2109 - val_accuracy: 0.9239
Epoch 6/15
57611/57611 [==============================] - 16s 274us/sample - loss: 0.2307 - accuracy: 0.9173 - val_loss: 0.2149 - val_accuracy: 0.9231
Epoch 7/15
57611/57611 [=====================

A classifier program with such high accuracy suggests that the data can potentially be used to construct a more useful model, and we will attempt to produce a regression program to predict the actual points that a player will get in future.

Begin similarly to before by making a function to produce appropriate groups of data, ordered temporaly and containing the paramaters which will be used to make the prediction:

In [10]:
%load_ext tensorboard

In [32]:
!rm -rf ./logs/ 

In [66]:
def Sequence_data_i(passed_directory, length):
    j=0
    SEQ_LEN = length
    sequential_data = []  # this is a list that will CONTAIN the sequences
    for filename in os.listdir(f"{passed_directory}"):
        if filename.endswith(".csv"):
            df = pd.read_csv(f"{passed_directory}/{filename}", index_col="round")
            input_df = df[["value", "transfers_in", "transfers_out", 'bps', 'ict_index', 'minutes', "goals_scored", "goals_conceded",'future_points']]  # performance predictior network from 'bps', 'ict_index', 'minutes'
            prev_days = deque(maxlen=SEQ_LEN)  
            for i in input_df.values: 
                prev_days.append([n for n in i[:-1]])  
                if len(prev_days) == SEQ_LEN:  
                    sequential_data.append([np.array(prev_days), i[-1]])  
    random.shuffle(sequential_data)
    #print(np.shape(sequential_data))
    #normal_data = min_max_scaler.fit_transform(np.array(sequential_data))
    return sequential_data

Length = 3
x1 = []
Y2 = []
two= Sequence_data_i(working_dir, Length)
for features, result in two:
    x1.append(features)
    Y2.append(result)

x1 = np.asarray(x1)
Y2 = np.asarray(Y2)

The x1 array contains the input paramaters and Y2 the corresponding output paramaters. Now the network can be trained using this data:

In [99]:
import tensorflow as tf
import datetime
from tensorflow.keras.optimizers import SGD
DropOut = 0.2
def Network_ii(IN, OUT, TIME_PERIOD, EPOCHS, BATCH_SIZE, LTSM_SHAPE):
 
    length = len(OUT)
    train_x = IN[:int(0.9 * length)]
    validation_x = IN[int(0.9 * length):]
    train_y = OUT[:int(0.9 * length)]
    validation_y = OUT[int(0.9 * length):]

    # Define Network & callback:
    train_x = train_x.reshape(train_x.shape[0],Length, 8)
    validation_x = validation_x.reshape(validation_x.shape[0],Length, 8)
    

    model = Sequential()
    model.add(LSTM(units=60, return_sequences= True, input_shape=(train_x.shape[1],8)))
    model.add(LSTM(LTSM_SHAPE, return_sequences=True))
    model.add(Dropout(DropOut))
    model.add(BatchNormalization())
    model.add(LSTM(units=30))
    model.add(Dense(units=1))
    LR=0.001
    opt = tf.keras.optimizers.Adamax(learning_rate=LR)
    model.compile(optimizer=opt, loss='mae')
    log_dir = "logs/fit/" + f"_Length={Length}_Dropout={DropOut}_LR={LR}LSTMax" 
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    train_y = np.asarray(train_y)
    validation_y = np.asarray(validation_y)
    history = model.fit(train_x,
                        train_y, 
                        batch_size=BATCH_SIZE,
                        epochs=EPOCHS,
                        validation_data=(validation_x, validation_y),
                        callbacks = [tensorboard_callback]
                       )
    print('\nhistory dict:', history.history)

    # Score model
    score = model.evaluate(validation_x, validation_y, verbose=0)
    print('Test loss:', score)
    # Save model
    model.save(f"models/{log_dir}")

EPOCHS, BATCH_SIZE, LTSM_SHAPE = 120, 32, 128

Network_ii(x1, Y2, Length, EPOCHS, BATCH_SIZE, LTSM_SHAPE)

Train on 12779 samples, validate on 1420 samples
Epoch 1/120
12779/12779 [==============================] - 6s 473us/sample - loss: 0.0557 - val_loss: 0.0745
Epoch 2/120
12779/12779 [==============================] - 3s 196us/sample - loss: 0.0442 - val_loss: 0.0350
Epoch 3/120
12779/12779 [==============================] - 3s 197us/sample - loss: 0.0329 - val_loss: 0.0443
Epoch 4/120
12779/12779 [==============================] - 3s 196us/sample - loss: 0.0205 - val_loss: 0.0167
Epoch 5/120
12779/12779 [==============================] - 3s 196us/sample - loss: 0.0143 - val_loss: 0.0189
Epoch 6/120
12779/12779 [==============================] - 3s 196us/sample - loss: 0.0111 - val_loss: 0.0135
Epoch 7/120
12779/12779 [==============================] - 3s 197us/sample - loss: 0.0083 - val_loss: 0.0119
Epoch 8/120
12779/12779 [==============================] - 3s 197us/sample - loss: 0.0081 - val_loss: 0.0072
Epoch 9/120
12779/12779 [==============================] - 3s 196us/sample - lo

12779/12779 [==============================] - 3s 202us/sample - loss: 0.0038 - val_loss: 0.0022
Epoch 76/120
12779/12779 [==============================] - 3s 199us/sample - loss: 0.0038 - val_loss: 0.0027
Epoch 77/120
12779/12779 [==============================] - 3s 198us/sample - loss: 0.0038 - val_loss: 0.0017
Epoch 78/120
12779/12779 [==============================] - 3s 204us/sample - loss: 0.0038 - val_loss: 0.0015
Epoch 79/120
12779/12779 [==============================] - 3s 201us/sample - loss: 0.0038 - val_loss: 0.0019
Epoch 80/120
12779/12779 [==============================] - 3s 199us/sample - loss: 0.0036 - val_loss: 0.0039
Epoch 81/120
12779/12779 [==============================] - 3s 198us/sample - loss: 0.0037 - val_loss: 0.0026
Epoch 82/120
12779/12779 [==============================] - 3s 206us/sample - loss: 0.0038 - val_loss: 0.0018
Epoch 83/120
12779/12779 [==============================] - 3s 208us/sample - loss: 0.0037 - val_loss: 0.0023
Epoch 84/120
12779/1277

Test loss: 0.0019862735230790477
INFO:tensorflow:Assets written to: models/logs/fit/_Length=3_Dropout=0.2_LR=0.001LSTMax/assets


In [92]:
player_list = pd.read_csv(f'raw_data/2019-20/player_idlist.csv')
time_period = Length
pred_week = 15
paramaters =["value", "transfers_in", "transfers_out", 'bps', 'ict_index', 'minutes', "goals_scored", "goals_conceded"]
actual_performance = ['future_points']
print("loading")
mod = tf.keras.models.load_model("models/new_model")
print("loaded")
facts = normlisation_factors
errs=[]
totalerror=0
TOT =0
TOTAV = 0
for i in range(len(player_list)):
    player = pd.read_csv(f'raw_data/2019-20/players/{player_list.iloc[i][0]}_{player_list.iloc[i][1]}_{player_list.iloc[i][2]}/gw.csv', index_col='round')
    
    try:
        if player["bps"].iloc[11] > 0:
            print(f"{player_list.iloc[i][0]}_{player_list.iloc[i][1]}:")
            for var, value in facts.items():
                try:
                    player[var]= player[var].div(value)
                except:
                    KeyError: 'attempted_passes'
                    pass
            try:
                player_in = player[paramaters].iloc[pred_week - time_period:pred_week]
                IN = np.array(player_in.values)
                IN_reshaped = IN.reshape(1,Length, 8)
                prediction = mod.predict(IN_reshaped)
                print("actual:", facts["bps"]*player["bps"].iloc[pred_week])
                print("predicted:", facts["bps"]*prediction[0][0])
                totalerror += ((facts["bps"]*player["bps"].iloc[pred_week]-facts["bps"]*prediction[0][0])**2)**0.5
                TOT +=1
                TOTAV += facts["bps"]*player["bps"].iloc[pred_week]
            except (ValueError, IndexError) as e:
                print(e)
                errs.append(f"{player_list.iloc[i][0]}_{player_list.iloc[i][1]}_{player_list.iloc[i][2]}")
    
    except (ValueError, IndexError) as e:
        errs.append(f"{player_list.iloc[i][0]}_{player_list.iloc[i][1]}_{player_list.iloc[i][2]}")

loading
loaded
Héctor_Bellerín:
actual: 0.0
predicted: 0.14410091936588287
Sead_Kolasinac:
actual: 26.0
predicted: 13.792533069849014
Rob_Holding:
actual: 0.0
predicted: 0.051655903458595276
Pierre-Emerick_Aubameyang:
actual: 44.0
predicted: 57.82554316520691
Alexandre_Lacazette:
actual: 0.0
predicted: 1.026152417063713
Bernd_Leno:
actual: 19.0
predicted: 20.247269332408905
Mesut_Özil:
actual: 9.0
predicted: 26.097900688648224
Lucas_Torreira:
actual: 18.0
predicted: 4.128911733627319
Matteo_Guendouzi:
actual: 3.0
predicted: 13.117479756474495
David_Luiz Moreira Marinho:
actual: 0.0
predicted: 15.100867867469788
Calum_Chambers:
actual: 14.0
predicted: 6.021416813135147
Nicolas_Pépé:
actual: 33.0
predicted: 0.0891571193933487
Joseph_Willock:
actual: 0.0
predicted: 8.92178575694561
Neil_Taylor:
actual: 0.0
predicted: 0.050873637199401855
Ørjan_Nyland:
actual: 0.0
predicted: 0.04995886981487274
Anwar_El Ghazi:
actual: 4.0
predicted: 15.215150088071823
John_McGinn:
actual: 10.0
predicted: 1

Aaron_Wan-Bissaka:
actual: 24.0
predicted: 10.966705352067947
Harry_Maguire:
actual: 16.0
predicted: 8.0806635171175
Victor_Lindelöf:
actual: 18.0
predicted: 25.12847149372101
Marcos_Rojo:
actual: 0.0
predicted: 0.05421334505081177
Marcus_Rashford:
actual: 28.0
predicted: 7.220625042915344
Mason_Greenwood:
actual: 0.0
predicted: 3.19131763279438
David_de Gea:
actual: 13.0
predicted: 10.102568686008453
Anthony_Martial:
actual: 27.0
predicted: 1.0445046722888947
Jesse_Lingard:
actual: 1.0
predicted: 4.109310485422611
Daniel_James:
actual: 19.0
predicted: 7.249440640211105
Frederico_Rodrigues de Paula Santos:
actual: 14.0
predicted: 10.033548340201378
Andreas_Pereira:
actual: 2.0
predicted: 14.0774427652359
Scott_McTominay:
actual: 10.0
predicted: 0.05185890197753906
DeAndre_Yedlin:
actual: 0.0
predicted: 0.1438308209180832
Federico_Fernández:
actual: 24.0
predicted: 17.018291652202606
Paul_Dummett:
actual: 10.0
predicted: 13.93639361858368
Jamaal_Lascelles:
actual: 0.0
predicted: 0.05394

The mean root squared error for this model is:

In [100]:
totalerror/len(player_list)

3.9169484039541125

The average number of point for a player in the data set is:

In [101]:
TOTAV/TOT

10.832699619771864

Thus the average error of the regression model is:

In [103]:
(totalerror/(len(player_list)))/(TOTAV/TOT)

0.36158561959983554

As the predictions above show, the model gives reasonably good predictions. Given the range of points that a player can achieve, although there is a significant error for some players, this is still useful metric, as the model tends to predict high scores for players who did score highly and vice versa. Due to the random nature of sport it is not surpising that there are a number of outliers however. Further invesigation into the nature of these outliers ( such as measuring the statistics of the error, eg, modelling it to a gaussian distribution)
would be useful in determining the exact nature of this model- it perhaps gives a good baseline prediction of players' performance, but does not allow for the random intricacies of the sport.

In [77]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 97761), started 1:46:31 ago. (Use '!kill 97761' to kill it.)